In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

from pipeline2 import H5DataReader
from pipeline2.detection import SimpleSingleChannelSpotDetector
from pipeline2.taskgeneration import BoundingBoxLocationGrouper
from pipeline2.util import filter_dict

%matplotlib notebook
plt.rcParams['figure.figsize'] = [7,7]

def blend(img, cmap, contrast=8, alpha=.8):
    
    norm = plt.Normalize()(img)
    blend = cmap(norm)
    
    # old alpha (everything below mode transparent, everything above alpha)
    #freq, edges = np.histogram(img.ravel(), bins=100)
    #mode = edges[np.argmax(freq)+1]
    #blend[:,:,-1] = (img > mode) * alpha
    
    blend[:,:,-1] = norm**(1/contrast) * alpha
    return blend


def do_h5_visualization(path):
    with h5py.File(path, 'r') as fd:
        plt.figure()    
        r = H5DataReader(fd)
        #r2 = type('',(object,),{'get_data':lambda self : r.get_data(idx)})()
        #det = SimpleSingleChannelSpotDetector(r2, 2, 1.5, 0, 3, 5, False).withPlotDetections().withVerbose()
        #BoundingBoxLocationGrouper(det, [5e-6, 5e-6, 2e-6]).withVerbose().get_locations()

        cms = [plt.cm.Reds_r, plt.cm.Greens_r, plt.cm.Blues_r]
        cols = ['red', 'green', 'blue']

        for idx in r.idxes:

            rd = r.get_data(idx)
            dta = rd.data


            meta = rd.measurementSettings[0]
            offsOldStg = np.array([filter_dict(
                meta, 'ExpControl/scan/range/offsets/coarse/{}/g_off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            offsOldG = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/g_off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            offsOld = offsOldStg + offsOldG + np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            lensOld = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/len'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            pszOld = np.array([filter_dict(
                    meta, 'ExpControl/scan/range/{}/psz'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)

            #left, right, bottom, top
            left = offsOld[0] - (lensOld[0] + pszOld[0]) /2
            right = offsOld[0] + (lensOld[0] + pszOld[0])/2
            bottom = offsOld[1] + (lensOld[1] + pszOld[1])/2
            top = offsOld[1] - (lensOld[1] + pszOld[1])/2

            # project and show
            img = np.squeeze(np.apply_along_axis(np.max, 1, dta[0][0]))
            plt.imshow(blend(img, cms[len(idx)-1]), extent=(left, right, bottom, top))

            # TODO: keep track of min/max and zoom out
            #plt.xlim(0,20)
            #plt.ylim(0,20)

            ax = plt.gca()
            ax.add_artist(Rectangle((left, top), right-left, bottom-top, fill=False, color=cols[len(idx)-1]))

        plt.gca().patch.set_facecolor('black')
    

In [ ]:
path = '/Volumes/cooperation_data/TobiasRagoczy_StamLab/DavidHoerl/20180226_pipeline2test/7d0fcf9627909e8b1afa42d73edbf0fd.h5'

do_h5_visualization(path)